Preparing data for analysis

In [ ]:
# uncomment + run if they aren't installed already
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

y_gpu_energies = []
d_gpu_energies = []

y_cpu_energies = []
d_cpu_energies = []

# iterate over 30 runs per model
for i in range(30):
    df_yolo = pd.read_csv(f'../experiment/results/yolo_run{i+1}.csv')
    df_detr = pd.read_csv(f'../experiment/results/rfdet_run{i+1}.csv')

    # convert power to Watts
    y_gpu_power = df_yolo['GPU0_POWER (mWatts)'] / 1000
    d_gpu_power = df_detr['GPU0_POWER (mWatts)'] / 1000
    
    # make time start from zero
    y_time = df_yolo['Time'] - df_yolo['Time'].iloc[0]
    d_time = df_detr['Time'] - df_detr['Time'].iloc[0]
    
    # compute total energy of this run
    # time is converted to seconds
    y_gpu_energy = np.trapz(y_gpu_power, y_time / 1000)
    d_gpu_energy = np.trapz(d_gpu_power, d_time / 1000)
    
    # append energy per model execution
    # 1500 images assumed per run 
    y_gpu_energies.append(y_gpu_energy / 1500)
    d_gpu_energies.append(d_gpu_energy / 1500)
    
    # monotonic counter -> need to compute difference between start and end for total energy
    y_cpu_energy = df_yolo['PP0_ENERGY (J)'].iloc[-1] - df_yolo['PP0_ENERGY (J)'].iloc[0]
    d_cpu_energy = df_detr['PP0_ENERGY (J)'].iloc[-1] - df_detr['PP0_ENERGY (J)'].iloc[0]
    
    y_cpu_energies.append(y_cpu_energy / 1500)
    d_cpu_energies.append(d_cpu_energy / 1500)

Violin and Box plots

In [ ]:
def plot_violin_and_box(yolo, detr, y_axis, y_label, title):

    data = pd.DataFrame({
        y_axis: np.concatenate([yolo, detr]),
        'Model': ['YOLO']*len(yolo) + ['DETR']*len(detr)
    })
    
    # Violin + Box plots
    sns.violinplot(data=data, x='Model', y=y_axis, inner='box')
    plt.xticks(ticks=[0, 1], labels=labels)
    plt.ylabel(y_label)
    plt.title(title)
    
    plt.show()
    
labels = ['YOLO', 'RF-DETR']

plot_violin_and_box(y_cpu_energies, d_cpu_energies, 'Energy', 'Energy Consumption (Joules)', 'CPU Energy Consumption: YOLO vs RF-DETR')

# plot_violin_and_box(y_gpu_power, d_gpu_power, 'Power', 'Power Consumption (Watts)', 'GPU Power Consumption: YOLO vs RF-DETR')

plot_violin_and_box(y_gpu_energies, d_gpu_energies, 'Energy', 'Energy Consumption (Joule)', 'GPU Energy Consumption: YOLO vs RF-DETR')

Statistical tests

In [ ]:
# !pip install scipy

from scipy.stats import ttest_ind, shapiro, mannwhitneyu


# if data is not normal (see the violin plots), then we need other tests / procedures

def statistical_tests(y, d):
    # Shapiro-Wilk test (testing normality)
    shap_yolo = shapiro(y.values)
    shap_detr = shapiro(d.values)

    # Statistical significance tests depending on normality
    if shap_yolo.pvalue < 0.05 or shap_detr.pvalue < 0.05:
        print("Some dataset is NOT normally distributed.")

        # Statistical significance test
        print('Statistical significance test')

        # null hypothesis: distribution of yolo is the same as rf-detr
        # if p_values < 0.05, then we can assume different distributions
        mwu = mannwhitneyu(y.values, d.values)
        print(f'\tMann-Whitney U test: '
              f'\n\t\t{mwu.statistic} '
              f'\n\t\t{mwu.pvalue}')

        print('Effect size analysis')

        median_yolo = np.median(y.values)
        median_detr = np.median(d.values)
        median_difference = median_yolo - median_detr

        cles = mwu.statistic / (len(y.values) * len(d.values))

        print(f'\tMedian difference: {median_difference:.2f}')
        print(f'\tCommon language effect size: {cles:.2f}')

    else:
        print("Normal distribution can be assumed.")

        # Statistical significance test (Welch's t-test)
        _, welch_pvalue = ttest_ind(
            y.values, d, equal_var=False, alternative='two-sided')

        print('\nStatistical significance test')
        if welch_pvalue < 0.05:
            print(f'\tP-value is significant ({welch_pvalue[0]})')
        else:
            print(f'\tP-value is not significant ({welch_pvalue[0]})')

        print('\nEffect size analysis')

        # using "raw" difference to retain more information
        mean_diff = y.values.mean() - d.values.mean()
        percentage_change = (mean_diff / y.values.mean()) * 100
        cohen_d = mean_diff / (0.5 * np.sqrt(np.square(y.values.std()) + np.square(d.values.std())))

        print(f'\tMean difference: {mean_diff:.2f}')
        print(f'\tPercentage change: {percentage_change:.2f}')
        print(f'\tCohen\'s Difference: {cohen_d:.2f}')


print('GPU energy consumption')
statistical_tests(np.array(y_gpu_energies), np.array(d_gpu_energies))
print('-------')
print('CPU energy consumption')
statistical_tests(y_cpu_energies, d_cpu_energies)

Graph plots

In [ ]:
# Nvidia GPU label: GPU0_POWER (mWatts)

# CPU

plt.plot(range(30), y_cpu_energies, label='YOLO')
plt.plot(range(30), d_cpu_energies, label='RF-DETR')

plt.xlabel('Number of runs')
plt.ylabel('CPU Energy Consumption (J)')
plt.title('CPU Energy Consumption')
plt.legend()
plt.grid()
plt.show()

# GPU

plt.plot(range(30), y_gpu_energies, label='YOLO')
plt.plot(range(30), d_gpu_energies.values, label='RF-DETR')

plt.xlabel('Number of runs')
plt.ylabel('GPU Energy Consumption (J)')
plt.title('GPU Energy Consumption')
plt.legend()
plt.grid()
plt.show()